In [2]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, GRU, concatenate, Bidirectional 
from keras import Model
from keras import optimizers
from tensorflow import keras

In [3]:
data_full = pd.read_csv('Intraday Volatility Dataset.csv')

In [4]:
X = data_full[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]]
Y = data_full["target"]
data_set = data_full[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility", "target"]]

In [5]:
splitlimit = int(len(data_set)*0.8)
training_features, test_data = data_set[:splitlimit], data_set[splitlimit:]

In [6]:
#Outlier removal

training_features["volatility_rolling_median"] = training_features["Daily Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features["return_rolling_median"] = training_features["Return"].rolling(window=41, center=True, min_periods=1).median()
training_features["EMAF_rolling_median"] = training_features["EMAF"].rolling(window=41, center=True, min_periods=1).median()
training_features["volume_rolling_median"] = training_features["Volume"].rolling(window=41, center=True, min_periods=1).median()

training_features["volatility minus median"] = (training_features["Daily Volatility"] - training_features["volatility_rolling_median"]).abs()
training_features["return_squared minus median"] = (training_features["Return_Squared"] - training_features["return_squared_rolling_median"]).abs()
training_features["return minus median"] = (training_features["Return"] - training_features["return_rolling_median"]).abs()
training_features["EMAF minus median"] = (training_features["EMAF"] - training_features["EMAF_rolling_median"]).abs()
training_features["volume minus median"] = (training_features["Volume"] - training_features["volume_rolling_median"]).abs()

volatility_outliers_removed = training_features[~(training_features['volatility minus median'] > 3 * training_features['volatility minus median'].median())]
all_outliers_removed = volatility_outliers_removed[~(volatility_outliers_removed['return_squared minus median'] > 3 * volatility_outliers_removed['return_squared minus median'].median())]
all_outliers_removed = all_outliers_removed[~(all_outliers_removed['return minus median'] > 3 * volatility_outliers_removed['return minus median'].median())]
all_outliers_removed = all_outliers_removed[~(all_outliers_removed['EMAF minus median'] > 3 * volatility_outliers_removed['EMAF minus median'].median())]
all_outliers_removed = all_outliers_removed[~(all_outliers_removed['volume minus median'] > 3 * volatility_outliers_removed['volume minus median'].median())]

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_21572/1273127160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_features["volatility_rolling_median"] = training_features["Daily Volatility"].rolling(window=41, center=True, min_periods=1).median()
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_21572/1273127160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=41, cente

In [7]:
X_cleaned = all_outliers_removed[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]]
Y_cleaned = all_outliers_removed["target"]
data_set_cleaned = all_outliers_removed[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility", "target"]]

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
training_data_features_scaled = scaler.fit_transform(X_cleaned)
data_set_scaled = scaler.fit_transform(data_set_cleaned)

In [8]:
#Reconstructing training data for logistic regression
Z = []

backcandles = 10

for j in range(5):
    Z.append([])
    for i in range(backcandles, training_data_features_scaled.shape[0]):
        Z[j].append(training_data_features_scaled[i-backcandles:i, j])
        
Z = np.moveaxis(Z, [0], [2])        
Z, yi = np.array(Z), np.array(data_set_scaled[backcandles-1:, -1])
y_final = np.reshape(yi,(len(yi),1))
y_final = y_final[1:]

In [9]:
test_data_features_scaled = scaler.fit_transform(test_data[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]])
test_dataset_scaled = scaler.fit_transform(test_data)

In [10]:
#Reconstructing test data for logistic regression
T = []

backcandles = 10

for j in range(5):
    T.append([])
    for i in range(backcandles, test_data_features_scaled.shape[0]):
        T[j].append(test_data_features_scaled[i-backcandles:i, j])
        
        
T = np.moveaxis(T, [0], [2])   
T, yi_test = np.array(T), np.array(test_dataset_scaled[backcandles-1:, -1])
y_final_test = np.reshape(yi_test,(len(yi_test),1))
y_final_test = y_final_test[1:]

In [12]:
#Logistic Regression model
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from matplotlib import cm
import numpy as np
from sklearn.metrics import confusion_matrix

Z_flattened = Z.reshape(1186, -1)
y_final_flattened = y_final.ravel()

T_flattened = T.reshape(945, -1)
y_final_test_flattened = y_final_test.ravel()


model_LR = LogisticRegression()
model_LR.fit(Z_flattened, y_final_flattened)


test_predictions1 = model_LR.predict(T_flattened)
dataframe = pd.DataFrame(y_final_test_flattened, columns = ["target"])
dataframe["predicted"] = test_predictions1
cm_LR = confusion_matrix(dataframe['predicted'], dataframe['target'])
print(cm_LR)



fpr, tpr, thresholds = roc_curve(dataframe['target'], dataframe['predicted'])
roc_auc = auc(fpr, tpr)
print(roc_auc)

[[325 146]
 [161 313]]
0.6753207455821835


In [13]:
#FFNN model
input_shape = len(training_data_features_scaled[0, :]) 

ff_input = Input(shape=input_shape, name='ff_input')
inputs = Dense(80, activation='relu', name='first_layer')(ff_input)  
outputs = Dense(1, activation='sigmoid', name='output')(inputs)
model = Model(inputs=ff_input, outputs=outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(x=training_data_features_scaled, y=data_set_scaled[:, -1], epochs=50, validation_data=(training_data_features_scaled, data_set_scaled[:, -1]))

Epoch 1/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6896 - accuracy: 0.5619 - val_loss: 0.6852 - val_accuracy: 0.5619
Epoch 2/50
38/38 [==============================] - 0s 520us/step - loss: 0.6826 - accuracy: 0.5619 - val_loss: 0.6792 - val_accuracy: 0.5619
Epoch 3/50
38/38 [==============================] - 0s 425us/step - loss: 0.6771 - accuracy: 0.5619 - val_loss: 0.6735 - val_accuracy: 0.5761
Epoch 4/50
 1/38 [..............................] - ETA: 0s - loss: 0.6763 - accuracy: 0.5312

2024-04-19 18:27:59.293811: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2024-04-19 18:27:59.293994: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


38/38 [==============================] - 0s 413us/step - loss: 0.6722 - accuracy: 0.5811 - val_loss: 0.6684 - val_accuracy: 0.5870
Epoch 5/50
38/38 [==============================] - 0s 466us/step - loss: 0.6668 - accuracy: 0.5920 - val_loss: 0.6633 - val_accuracy: 0.6020
Epoch 6/50
38/38 [==============================] - 0s 458us/step - loss: 0.6621 - accuracy: 0.6112 - val_loss: 0.6585 - val_accuracy: 0.6229
Epoch 7/50
38/38 [==============================] - 0s 463us/step - loss: 0.6572 - accuracy: 0.6129 - val_loss: 0.6539 - val_accuracy: 0.6296
Epoch 8/50
38/38 [==============================] - 0s 470us/step - loss: 0.6528 - accuracy: 0.6288 - val_loss: 0.6490 - val_accuracy: 0.6363
Epoch 9/50
38/38 [==============================] - 0s 438us/step - loss: 0.6476 - accuracy: 0.6329 - val_loss: 0.6448 - val_accuracy: 0.6338
Epoch 10/50
38/38 [==============================] - 0s 447us/step - loss: 0.6435 - accuracy: 0.6371 - val_loss: 0.6409 - val_accuracy: 0.6338
Epoch 11/50
38/3

In [14]:
FFNN_predictions = model.predict(test_data_features_scaled)
FFNN_predictions_classes = (FFNN_predictions > 0.5).astype(int)
y_frame_FFNN = test_dataset_scaled[:, -1]

df = pd.DataFrame({
    "target": y_frame_FFNN,
    "predicted": FFNN_predictions_classes.ravel()  
})

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df['predicted'], df['target'])
print(cm)

[[439 328]
 [ 51 137]]


In [16]:
fpr, tpr, thresholds = roc_curve(df['target'], df['predicted'])
roc_auc = auc(fpr, tpr)
roc_auc

0.5952710116304587